# Passing Networks

In [1]:
from mplsoccer import Sbopen,Pitch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [11]:
parser = Sbopen()
df,related,freeze,tactics = parser.event(18244)

## Preparing the data
- Filter: only successful passes until the first sub
- Need: Pass maker and Receiver. Start and End locations

In [24]:
df.columns

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'possession',
       'duration', 'match_id', 'type_id', 'type_name', 'possession_team_id',
       'possession_team_name', 'play_pattern_id', 'play_pattern_name',
       'team_id', 'team_name', 'tactics_formation', 'player_id', 'player_name',
       'position_id', 'position_name', 'pass_recipient_id',
       'pass_recipient_name', 'pass_length', 'pass_angle', 'pass_height_id',
       'pass_height_name', 'end_x', 'end_y', 'sub_type_id', 'sub_type_name',
       'body_part_id', 'body_part_name', 'x', 'y', 'outcome_id',
       'outcome_name', 'under_pressure', 'counterpress', 'off_camera',
       'foul_won_defensive', 'pass_switch', 'dribble_overrun', 'pass_cross',
       'pass_assisted_shot_id', 'pass_shot_assist', 'shot_statsbomb_xg',
       'end_z', 'shot_key_pass_id', 'technique_id', 'technique_name',
       'goalkeeper_position_id', 'goalkeeper_position_name', 'pass_backheel',
       'foul_committed_advantage', 'foul_won_

In [13]:
df.type_name.unique()

array(['Starting XI', 'Half Start', 'Pass', 'Ball Receipt', 'Carry',
       'Clearance', 'Pressure', 'Block', 'Dribbled Past', 'Dribble',
       'Foul Committed', 'Foul Won', 'Miscontrol', 'Ball Recovery',
       'Dispossessed', 'Duel', 'Shot', 'Goal Keeper', 'Interception',
       'Offside', 'Tactical Shift', 'Injury Stoppage', 'Half End',
       'Bad Behaviour', 'Substitution', '50/50'], dtype=object)

In [16]:
# check for index for the first sub
sub = df.loc[df['type_name']=='Substitution'].loc[df['team_name']=='Real Madrid'].iloc[0]['index']
# make df with successful passes by England til the first substition
mask_rm = (df.type_name=='Pass') & (df.team_name=='Real Madrid') & (df.index<sub) & (df.outcome_name.isnull()) & (df.sub_type_name !='Throw-in')
# taking necessary columns
df_pass = df.loc[mask_rm,['x','y','end_x','end_y','player_name','pass_recipient_name']]
# adjusting that only the surname of a player is presented
df_pass['player_name'] = df_pass['player_name'].apply(lambda x:str(x).split()[-1])
df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])

In [18]:
df_pass.head()

,x,y,end_x,end_y,player_name,pass_recipient_name
16,29.0,73.0,32.0,74.0,Ramos,Modrić
55,15.0,54.0,21.0,62.0,Kroos,Modrić
64,27.0,56.0,41.0,63.0,Varane,Modrić
93,48.0,70.0,57.0,65.0,Varane,Kroos
96,57.0,65.0,61.0,75.0,Kroos,Modrić


## Calculating vertices size and location

- location: every player's average location
- vertices size: number of passes made by each player

In [23]:
scatter_df = pd.DataFrame()
for i, name in enumerate(df_pass['player_name'].unique()):
    # print(i,name)
    passx = df_pass.loc[df_pass['player_name']==name]['x'].to_numpy()
    recx = df_pass.loc[df_pass["pass_recipient_name"] == name]["end_x"].to_numpy()
    passy = df_pass.loc[df_pass['player_name']==name]['y'].to_numpy()
    recy = df_pass.loc[df_pass["pass_recipient_name"] == name]["end_y"].to_numpy()
    scatter_df.at[i, "player_name"] = name
    scatter_df.at[i, "x"] = np.mean(np.concatenate([passx, recx]))
    scatter_df.at[i, "y"] = np.mean(np.concatenate([passy, recy]))
    #calculate number of passes
    scatter_df.at[i, "num"] = df_pass.loc[df_pass["player_name"] == name].count().iloc[0]

# adjust circle size for vertices
scatter_df['marker_size'] = (scatter_df['num'] / scatter_df['num'].max() * 1500)


0 Ramos
1 Kroos
2 Varane
3 Modrić
4 García
5 Júnior
6 Suárez
7 Gamboa
8 Casimiro
9 Aveiro
10 Benzema


## Calculating edges width
